In [1]:
import pandas as pd

df = pd.read_csv('bigBasketProducts.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27555 entries, 0 to 27554
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         27555 non-null  int64  
 1   product       27554 non-null  object 
 2   category      27555 non-null  object 
 3   sub_category  27555 non-null  object 
 4   brand         27554 non-null  object 
 5   sale_price    27555 non-null  float64
 6   market_price  27555 non-null  float64
 7   type          27555 non-null  object 
 8   rating        18929 non-null  float64
 9   description   27440 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 2.1+ MB


In [3]:
df.fillna("Not Available", inplace=True)

In [4]:
df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [5]:
df.columns

Index(['index', 'product', 'category', 'sub_category', 'brand', 'sale_price',
       'market_price', 'type', 'rating', 'description'],
      dtype='object')

In [1]:
from meilisearch import Client

client = Client('http://localhost:7700')

In [3]:
a = client.get_indexes({'limit': 3})

In [6]:
client.index('movies').delete()

TaskInfo(task_uid=57346, index_uid='movies', status='enqueued', type='indexDeletion', enqueued_at=datetime.datetime(2024, 4, 14, 8, 19, 12, 753447))

In [41]:
client.create_index('products',  {'primaryKey': 'index'})

TaskInfo(task_uid=27562, index_uid='products', status='enqueued', type='indexCreation', enqueued_at=datetime.datetime(2024, 4, 12, 12, 41, 8, 278427))

In [8]:
data = df.to_dict(orient='records' )

In [9]:
data = [{str(k): v for k, v in item.items()} for item in data]

In [42]:
data[0]

{'index': 1,
 'product': 'Garlic Oil - Vegetarian Capsule 500 mg',
 'category': 'Beauty & Hygiene',
 'sub_category': 'Hair Care',
 'brand': 'Sri Sri Ayurveda ',
 'sale_price': 220.0,
 'market_price': 220.0,
 'type': 'Hair Oil & Serum',
 'rating': 4.1,
 'description': 'This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/'}

In [44]:
for d in data:
    client.index('products').add_documents([d], primary_key='index')

In [2]:
index = client.get_index('products')

In [4]:
x = index.search('Garlic Oil - Vegetarian Capsule 500 mg') 

In [15]:
x = index.search( 'smart tech for home', {
  'limit': 3,
  "hybrid": {
      "semanticRatio": 0.5,
      "embedder": "default"
    }
})

In [16]:
x

{'hits': [{'product': 'Smart LED Bulb - 12 Watt, Wi-Fi Enabled, Multicolour (SSK-SMW 12W)',
   'category': 'Kitchen, Garden & Pets',
   'sub_category': 'Appliances & Electricals',
   'brand': 'Syska',
   'type': 'CFL & Led Bulbs',
   'sale_price': 749.0,
   'market_price': 2499.0,
   'rating': 3.2,
   'description': 'Too lazy to turn on or turn off your light? Syska Smart Bulb has got your back. It comes equipped with features such as Voice Control and Wi-Fi Compatibility so that you can control the light with just your voice. Wifi-enabled, With the alarm clock feature in this smart bulb, you can easily set daily schedules.\xa0Syska smart bulb works perfectly well with any Wi-Fi router without a need for a separate hub or paid subscription service. Your existing Wi-Fi is enough (2.4 GHz only). It works well with a wireless connection. It has 16 million colour options as it offers you 16 million shades and different tones to make any occasion extra special and allow to change colours as

In [7]:
index.get_settings()

{'displayedAttributes': ['product',
  'category',
  'sub_category',
  'brand',
  'sale_price',
  'market_price',
  'type',
  'rating',
  'description'],
 'searchableAttributes': ['product',
  'category',
  'sub_category',
  'brand',
  'type'],
 'filterableAttributes': ['Sale_price',
  'brand',
  'category',
  'rating',
  'sub_category'],
 'sortableAttributes': ['market_price', 'rating', 'sale_price'],
 'rankingRules': ['words',
  'typo',
  'proximity',
  'attribute',
  'sort',
  'exactness'],
 'stopWords': [],
 'nonSeparatorTokens': [],
 'separatorTokens': [],
 'dictionary': [],
 'synonyms': {},
 'distinctAttribute': None,
 'proximityPrecision': 'byWord',
 'typoTolerance': {'enabled': True,
  'minWordSizeForTypos': {'oneTypo': 5, 'twoTypos': 9},
  'disableOnWords': [],
  'disableOnAttributes': []},
 'faceting': {'maxValuesPerFacet': 100, 'sortFacetValuesBy': {'*': 'alpha'}},
 'pagination': {'maxTotalHits': 300},
 'embedders': {'default': HuggingFaceEmbedder(source='huggingFace', model=

In [54]:
index.update_displayed_attributes(['product', 'category', 'sub_category', 'brand', 'sale_price', 'market_price', 'type', 'rating', 'description'])

TaskInfo(task_uid=57336, index_uid='products', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2024, 4, 12, 13, 44, 28, 647563))

In [55]:
index.get_distinct_attribute()

In [56]:
index.get_filterable_attributes()

[]

In [57]:
index.update_filterable_attributes(['category', 'sub_category', 'brand', 'Sale_price', 'rating'])

TaskInfo(task_uid=57337, index_uid='products', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2024, 4, 12, 13, 57, 54, 845662))

In [59]:
index.get_filterable_attributes()

['Sale_price', 'brand', 'category', 'rating', 'sub_category']

In [60]:
index.get_pagination_settings()

Pagination(max_total_hits=1000)

In [61]:
index.update_pagination_settings({'maxTotalHits': 300})

TaskInfo(task_uid=57338, index_uid='products', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2024, 4, 12, 14, 2, 0, 516406))

In [62]:
index.get_searchable_attributes()

['*']

In [63]:
index.update_searchable_attributes(['product', 'category', 'sub_category', 'brand', 'type'])

TaskInfo(task_uid=57339, index_uid='products', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2024, 4, 12, 14, 8, 35, 492149))

In [64]:
index.get_searchable_attributes()

['product', 'category', 'sub_category', 'brand', 'type']

In [65]:
index.get_sortable_attributes()

[]

In [66]:
index.update_sortable_attributes(['sale_price', 'market_price', 'rating'])

TaskInfo(task_uid=57340, index_uid='products', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2024, 4, 12, 14, 12, 37, 559498))

In [67]:
index.get_sortable_attributes()

['market_price', 'rating', 'sale_price']

In [68]:
index.get_stop_words()

[]

In [69]:
index.get_synonyms()

{}

In [70]:
index.get_typo_tolerance()

TypoTolerance(enabled=True, disable_on_attributes=[], disable_on_words=[], min_word_size_for_typos=MinWordSizeForTypos(one_typo=5, two_typos=9))